# Prerequisites
This guide supposes that TensorFlow and Docker are already installed.

# Preparing environment
My working directory is  “/Users/rabbah/Documents/projets/ai_training” change it with your working directory path
1. First I will go to my working directory
```bash
cd /Users/rabbah/Documents/projets/ai_training
```
2. After that we create a directory named “tf_files”
```bash
mkdir tf_files
cd tf_files/
```
3. At the “tf_files” directory level we will download a folder that contain some flowers images 
```bash
curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
tar xzf flower_photos.tgz
```
4. After that we install a TensorFlow image in Docker container, and we set the working directory of this image to the current folder:
```bash
docker run -it -v `pwd`:`pwd` -w `pwd` gcr.io/tensorflow/tensorflow:latest-devel
```
5. The TensorFlow image contain already a floder “/tensorflow” that contain all necessary dependencies (we execute git pull to retrieve some examples):
```bash
cd /tensorflow/
git pull
```
6. After that we set some variables that will help (feel free to change mydir variable to your adequate folder)
```bash
mydir=/Users/rabbah/Documents/projets/ai_training/tf_files
tfdir=/tensorflow/tensorflow/examples/image_retraining
cd $mydir
```
7. Now we lunch the training command:
```bash
python $tfdir/retrain.py --bottleneck_dir=$mydir/bottlenecks --how_many_training_steps 500 --model_dir=$mydir/inception --output_graph=$mydir/retrained_graph.pb --output_labels=$mydir/retrained_labels.txt --image_dir $mydir/flower_photos
```
After some minutes we got the following result:

![Training result](images/imgclassficationresult.png)

And if we list directory content we got:

![Directory content after training process](images/tf_files_ls.png)


# Testing our model:

In [3]:
import tensorflow as tf
import sys

def predict(image_path):
    
    # Read in the image_data
    image_data = tf.gfile.FastGFile(image_path, 'rb').read()

    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line 
                       in tf.gfile.GFile("/Users/rabbah/Documents/projets/ai_training/tf_files/retrained_labels.txt")]

    # Unpersists graph from file
    with tf.gfile.FastGFile("/Users/rabbah/Documents/projets/ai_training/tf_files/retrained_graph.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

    with tf.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

        predictions = sess.run(softmax_tensor, \
                 {'DecodeJpeg/contents:0': image_data})

        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]

        for node_id in top_k:
            human_string = label_lines[node_id]
            score = predictions[0][node_id]
            print('%s (score = %.5f)' % (human_string, score))


Lets give it a try :

In [5]:
predict('images/photo1.jpg')

daisy (score = 0.98994)
sunflowers (score = 0.00773)
dandelion (score = 0.00168)
tulips (score = 0.00060)
roses (score = 0.00005)


In [6]:
predict('images/photo2.jpg')

roses (score = 0.89137)
tulips (score = 0.10531)
sunflowers (score = 0.00177)
dandelion (score = 0.00123)
daisy (score = 0.00032)
